In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [18]:
data = pd.read_csv('2003_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Amapá - Produção de Cimento (t)'], axis=1)
data

,Ano,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Consumo de Cimento (t)
0,2003,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.035813e+06,356591.003430,10.883143,7.033593e+06,85.0930
1,2004,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.163242e+06,357271.494323,10.907898,7.078752e+06,80.0080
2,2005,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.284323e+06,359035.074957,10.930315,7.120175e+06,104.1970
3,2006,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.397333e+06,361349.457774,10.963941,7.156515e+06,115.5400
4,2007,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.489305e+06,362177.947424,10.970195,7.171045e+06,135.0460
5,2008,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.565394e+06,362905.280442,10.964829,7.173307e+06,116.3660
6,2009,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.619980e+06,360014.231547,10.929234,7.163325e+06,112.7290
7,2010,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.840356e+06,365333.788458,10.918810,7.227241e+06,151.5420
8,2011,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,9.053310e+06,369799.968255,10.934033,7.289555e+06,161.9700
9,2012,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.243435e+06,371290.647450,10.930025,7.338668e+06,261.1200


In [19]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.035813e+06,356591.003430,10.883143,7.033593e+06
1,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.163242e+06,357271.494323,10.907898,7.078752e+06
2,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.284323e+06,359035.074957,10.930315,7.120175e+06
3,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.397333e+06,361349.457774,10.963941,7.156515e+06
4,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.489305e+06,362177.947424,10.970195,7.171045e+06
5,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.565394e+06,362905.280442,10.964829,7.173307e+06
6,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.619980e+06,360014.231547,10.929234,7.163325e+06
7,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.840356e+06,365333.788458,10.918810,7.227241e+06
8,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,9.053310e+06,369799.968255,10.934033,7.289555e+06
9,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.243435e+06,371290.647450,10.930025,7.338668e+06


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
17    157.6210
18    142.7390
19         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [21]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-2.498491,-0.888884,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.651592,-0.411311,-0.183091,-1.880195
1,-1.725700,-0.907434,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.430423,-0.315776,0.264667,-1.501512
2,-1.057698,-0.915907,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.220272,-0.068185,0.670127,-1.154156
3,-0.493413,-0.938113,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.024129,0.256733,1.278344,-0.849422
4,-0.031772,-0.942515,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.864499,0.373045,1.391456,-0.727572
5,0.328298,-0.952835,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.732437,0.475156,1.294411,-0.708607
6,0.587870,-0.946844,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.637697,0.069279,0.650588,-0.792315
7,0.748015,-0.955637,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.255205,0.816097,0.462035,-0.256339
8,0.809807,-0.958964,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.114404,1.443108,0.737389,0.266205
9,0.547992,0.647212,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.444390,1.652386,0.664881,0.678053


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.821846,-1.025345,-1.444524,0.649078


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [25]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [26]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [27]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2478320502, 3115868966, 2495209344, 3619858874, 3625714342, 2649205072, 1862559630, 2590731765, 3244981117, 758394048, 2978537720, 3113167337, 314360819, 2689121080, 2403821583, 1071193047, 705769291, 1872644300, 690457683, 793731572, 2447830676, 2114005293, 2282667995, 987236667, 283294105, 1463376404, 4281254326, 373161264, 695168687, 59190186, 872269012, 2694056874, 2974441289, 4202015582, 3544738895, 1330052727, 1540323600, 837272133, 4273883471, 652881758, 3948956245, 81191944, 2336692055, 2582666968, 3460370894, 315299766, 1490984943, 1216041547, 2325641885, 58039327, 453640934, 88693678, 2337433216, 3147232815, 4120001362, 1767145742, 20014638, 518803383, 3623833622, 4051830210, 4272917340, 3593729183, 221200723, 2951922723, 4121588701, 1749416096, 543181456, 3791803048, 2157570661, 1796268008, 478909151, 4257051874, 2546427694, 1514900607, 1054256263, 128549830, 2511269313, 1811026490, 662040289, 4176367328, 3912083995, 2654530145, 1569997972, 2334534128, 2973224252, 356018295

val_loss: 1902.1474609375


Step: 84 ___________________________________________
val_loss: 2.8057405948638916


Step: 85 ___________________________________________
val_loss: 31.6114444732666


Step: 86 ___________________________________________
val_loss: 1975.94580078125


Step: 87 ___________________________________________
val_loss: 11.525843620300293


Step: 88 ___________________________________________
val_loss: 66.53479766845703


Step: 89 ___________________________________________
val_loss: 10.922526359558105


Step: 90 ___________________________________________
val_loss: 1.7990421056747437


Step: 91 ___________________________________________
val_loss: 6.481996059417725


Step: 92 ___________________________________________
val_loss: 1274.5203857421875


Step: 93 ___________________________________________
val_loss: 56.607364654541016


Step: 94 ___________________________________________
val_loss: 13.010684967041016


Step: 95 ___________________________________________
v

In [28]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 463ms/step - loss: 25824.1016 - val_loss: 12369.1045
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 22846.7969 - val_loss: 47417.5352
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 24832.7656 - val_loss: 7176.6719
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 17992.3125 - val_loss: 928.1135
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 15804.9229 - val_loss: 40042.7656
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 37251.2383 - val_loss: 8461.0029
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 25344.3945 - val_loss: 22497.9297
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 22786.0293 - val_loss: 17464.4082
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 22157.9141 - val_loss: 15509.2812
Epoch 10/10000
1/1 [===========

1/1 [==============================] - 0s 14ms/step - loss: 491.8195 - val_loss: 271.8426
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 400.5946 - val_loss: 227.5025
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 415.8185 - val_loss: 205.5828
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 394.9043 - val_loss: 252.6959
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 417.8201 - val_loss: 235.0000
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 466.6789 - val_loss: 151.8560
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 446.6255 - val_loss: 207.2505
Epoch 85/10000
1/1 [==============================] - 0s 14ms/step - loss: 423.1013 - val_loss: 182.1803
Epoch 86/10000
1/1 [==============================] - 0s 14ms/step - loss: 441.8223 - val_loss: 820.1668
Epoch 87/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 13ms/step - loss: 494.9368 - val_loss: 26.4154
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 420.4067 - val_loss: 50.1118
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 403.3224 - val_loss: 41.4944
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 407.3617 - val_loss: 38.1030
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 404.5576 - val_loss: 34.9803
Epoch 160/10000
1/1 [==============================] - 0s 14ms/step - loss: 400.3761 - val_loss: 163.1160
Epoch 161/10000
1/1 [==============================] - 0s 14ms/step - loss: 423.7198 - val_loss: 102.2012
Epoch 162/10000
1/1 [==============================] - 0s 14ms/step - loss: 402.3606 - val_loss: 88.9284
Epoch 163/10000
1/1 [==============================] - 0s 14ms/step - loss: 401.8141 - val_loss: 82.5513
Epoch 164/10000
1/1 [==============================] - 0s 14ms/step -

Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 293.1458 - val_loss: 46.8284
Epoch 234/10000
1/1 [==============================] - 0s 13ms/step - loss: 299.6494 - val_loss: 49.1333
Epoch 235/10000
1/1 [==============================] - 0s 14ms/step - loss: 356.5110 - val_loss: 41.1608
Epoch 236/10000
1/1 [==============================] - 0s 14ms/step - loss: 345.4305 - val_loss: 43.5492
Epoch 237/10000
1/1 [==============================] - 0s 14ms/step - loss: 345.4554 - val_loss: 42.4931
Epoch 238/10000
1/1 [==============================] - 0s 13ms/step - loss: 328.1235 - val_loss: 42.7274
Epoch 239/10000
1/1 [==============================] - 0s 13ms/step - loss: 336.1433 - val_loss: 42.2475
Epoch 240/10000
1/1 [==============================] - 0s 14ms/step - loss: 335.8372 - val_loss: 43.3622
Epoch 241/10000
1/1 [==============================] - 0s 14ms/step - loss: 377.6873 - val_loss: 39.5994
Epoch 242/10000
1/1 [==============================] - 

Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 211.0331 - val_loss: 56.0649
Epoch 312/10000
1/1 [==============================] - 0s 14ms/step - loss: 388.4383 - val_loss: 79.3870
Epoch 313/10000
1/1 [==============================] - 0s 14ms/step - loss: 346.9247 - val_loss: 83.0410
Epoch 314/10000
1/1 [==============================] - 0s 14ms/step - loss: 372.3523 - val_loss: 84.5484
Epoch 315/10000
1/1 [==============================] - 0s 13ms/step - loss: 364.4172 - val_loss: 80.8892
Epoch 316/10000
1/1 [==============================] - 0s 14ms/step - loss: 363.7313 - val_loss: 75.8766
Epoch 317/10000
1/1 [==============================] - 0s 14ms/step - loss: 354.3193 - val_loss: 55.7600
Epoch 318/10000
1/1 [==============================] - 0s 14ms/step - loss: 350.9100 - val_loss: 49.0445
Epoch 319/10000
1/1 [==============================] - 0s 14ms/step - loss: 345.7367 - val_loss: 46.8045
Epoch 320/10000
1/1 [==============================] - 

...

Epoch 1711/10000
1/1 [==============================] - 0s 13ms/step - loss: 61.3498 - val_loss: 3.7568
Epoch 1712/10000
1/1 [==============================] - 0s 14ms/step - loss: 62.9314 - val_loss: 3.5675
Epoch 1713/10000
1/1 [==============================] - 0s 13ms/step - loss: 70.1952 - val_loss: 4.9757
Epoch 1714/10000
1/1 [==============================] - 0s 14ms/step - loss: 60.5619 - val_loss: 4.2759
Epoch 1715/10000
1/1 [==============================] - 0s 13ms/step - loss: 60.8607 - val_loss: 3.8822
Epoch 1716/10000
1/1 [==============================] - 0s 14ms/step - loss: 60.9789 - val_loss: 3.5818
Epoch 1717/10000
1/1 [==============================] - 0s 13ms/step - loss: 60.9120 - val_loss: 3.4540
Epoch 1718/10000
1/1 [==============================] - 0s 14ms/step - loss: 60.0196 - val_loss: 3.1822
Epoch 1719/10000
1/1 [==============================] - 0s 14ms/step - loss: 99.5610 - val_loss: 3.5910
Epoch 1720/10000
1/1 [==============================] - 0s 13ms/

1/1 [==============================] - 0s 13ms/step - loss: 52.4924 - val_loss: 2.3391
Epoch 1790/10000
1/1 [==============================] - 0s 13ms/step - loss: 55.6742 - val_loss: 2.6844
Epoch 1791/10000
1/1 [==============================] - 0s 14ms/step - loss: 73.8405 - val_loss: 2.6033
Epoch 1792/10000
1/1 [==============================] - 0s 14ms/step - loss: 70.8441 - val_loss: 2.6325
Epoch 1793/10000
1/1 [==============================] - 0s 14ms/step - loss: 52.1448 - val_loss: 2.1765
Epoch 1794/10000
1/1 [==============================] - 0s 14ms/step - loss: 50.2541 - val_loss: 2.1042
Epoch 1795/10000
1/1 [==============================] - 0s 14ms/step - loss: 51.1108 - val_loss: 1.9597
Epoch 1796/10000
1/1 [==============================] - 0s 14ms/step - loss: 50.8855 - val_loss: 1.7910
Epoch 1797/10000
1/1 [==============================] - 0s 13ms/step - loss: 47.6789 - val_loss: 1.8435
Epoch 1798/10000
1/1 [==============================] - 0s 13ms/step - loss: 49.6

Epoch 1868/10000
1/1 [==============================] - 0s 14ms/step - loss: 60.3588 - val_loss: 0.8008
Epoch 1869/10000
1/1 [==============================] - 0s 13ms/step - loss: 62.0341 - val_loss: 0.9862
Epoch 1870/10000
1/1 [==============================] - 0s 14ms/step - loss: 61.2848 - val_loss: 0.8637
Epoch 1871/10000
1/1 [==============================] - 0s 14ms/step - loss: 63.6893 - val_loss: 43.3264
Epoch 1872/10000
1/1 [==============================] - 0s 13ms/step - loss: 68.1264 - val_loss: 28.4732
Epoch 1873/10000
1/1 [==============================] - 0s 13ms/step - loss: 65.8544 - val_loss: 10.2276
Epoch 1874/10000
1/1 [==============================] - 0s 14ms/step - loss: 64.8111 - val_loss: 10.8687
Epoch 1875/10000
1/1 [==============================] - 0s 13ms/step - loss: 65.6668 - val_loss: 9.1453
Epoch 1876/10000
1/1 [==============================] - 0s 14ms/step - loss: 65.4858 - val_loss: 8.6914
Epoch 1877/10000
1/1 [==============================] - 0s 1

1/1 [==============================] - 0s 14ms/step - loss: 90.2645 - val_loss: 0.3746
Epoch 1947/10000
1/1 [==============================] - 0s 14ms/step - loss: 89.9573 - val_loss: 0.3116
Epoch 1948/10000
1/1 [==============================] - 0s 14ms/step - loss: 89.6064 - val_loss: 0.3569
Epoch 1949/10000
1/1 [==============================] - 0s 13ms/step - loss: 89.2616 - val_loss: 0.5959
Epoch 1950/10000
1/1 [==============================] - 0s 14ms/step - loss: 87.6528 - val_loss: 0.6291
Epoch 1951/10000
1/1 [==============================] - 0s 13ms/step - loss: 95.5245 - val_loss: 0.3005
Epoch 1952/10000
1/1 [==============================] - 0s 13ms/step - loss: 108.4693 - val_loss: 1.5689
Epoch 1953/10000
1/1 [==============================] - 0s 14ms/step - loss: 80.5397 - val_loss: 1.1419
Epoch 1954/10000
1/1 [==============================] - 0s 13ms/step - loss: 85.1250 - val_loss: 0.5595
Epoch 1955/10000
1/1 [==============================] - 0s 13ms/step - loss: 110

1/1 [==============================] - 0s 13ms/step - loss: 42.6107 - val_loss: 2.5048
Epoch 2025/10000
1/1 [==============================] - 0s 14ms/step - loss: 57.6967 - val_loss: 1.9306
Epoch 2026/10000
1/1 [==============================] - 0s 13ms/step - loss: 41.6295 - val_loss: 1.8625
Epoch 2027/10000
1/1 [==============================] - 0s 13ms/step - loss: 44.6111 - val_loss: 1.7248
Epoch 2028/10000
1/1 [==============================] - 0s 14ms/step - loss: 41.0845 - val_loss: 1.7137
Epoch 2029/10000
1/1 [==============================] - 0s 13ms/step - loss: 59.0444 - val_loss: 3.1604
Epoch 2030/10000
1/1 [==============================] - 0s 13ms/step - loss: 41.4445 - val_loss: 2.0241
Epoch 2031/10000
1/1 [==============================] - 0s 14ms/step - loss: 41.3252 - val_loss: 2.2252
Epoch 2032/10000
1/1 [==============================] - 0s 14ms/step - loss: 41.6291 - val_loss: 1.7426
Epoch 2033/10000
1/1 [==============================] - 0s 14ms/step - loss: 40.3

In [29]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[157.621](test_target) - [[163.31464]](prediction) = 5.693636230468741


In [30]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [31]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-1.822242,1.867029,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.595141,-1.449699,-1.742940,-1.876401
1,-1.111250,1.045116,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.072139,-1.202688,-0.886900,-1.083744
2,-0.496667,0.669687,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.575192,-0.562528,-0.111729,-0.356662
3,0.022494,-0.314227,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.111371,0.277568,1.051082,0.281205
4,0.447218,-0.509271,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.266106,0.578301,1.267334,0.536260
5,0.778494,-0.966548,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.578397,0.842315,1.081801,0.575958
6,1.017308,-0.701091,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.802429,-0.207105,-0.149082,0.400740
7,1.164646,-1.090695,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.706911,1.723837,-0.509565,1.522643


train_target:


,Amapá - Consumo de Cimento (t)
0,80.008
1,104.197
2,115.540
3,135.046
4,116.366
5,112.729
6,151.542
7,161.970


test_input:


...

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.821846,-1.025345,-1.444524,0.649078


test_target:


,Amapá - Consumo de Cimento (t)
17,157.621


1/1 [==============================] - 0s 38ms/step
Error: 5.693636230468741




[135.53392028808594,
 171.2110595703125,
 183.6294708251953,
 230.3533172607422,
 207.08731079101562,
 201.67181396484375,
 162.6983184814453,
 126.78111267089844,
 165.54541015625,
 163.31463623046875]

In [32]:
display(targets)
display(predictions)

[261.12,
 250.519,
 227.243,
 198.0696,
 166.85039999999998,
 135.021,
 123.347,
 120.153,
 133.523,
 157.621]

[135.53392028808594,
 171.2110595703125,
 183.6294708251953,
 230.3533172607422,
 207.08731079101562,
 201.67181396484375,
 162.6983184814453,
 126.78111267089844,
 165.54541015625,
 163.31463623046875]

In [33]:
mae = mean_absolute_error(predictions, targets)
mae

47.137446887207034

In [34]:
porcentage = mae/np.mean(targets)
porcentage

0.2657926360468339